In [195]:
from __future__ import division

import numpy
import scipy
import scipy.stats
import numpy as np

On Twitter, @Asmodaie asked me this question: "If team A has a win rate of 20% and team B a win rate of 60% what are the odds of A winning from B?"

This is one of those probabiltiy questions that can't be answered simply by following the usual rules of probability theory. We're going to need to make some extra assumptions that we can try to make as reasonable as possible. I also don't pretend to know the best answer. I'm just going to give three approaches that are all somewhat heuristic and could easily be argued with.

Method 1 (Simulation)
========

If you know nothing about a pair of teams, you'd estimate that each team has a 0.5 chance of beating the other (assuming no draws.) So let's try to make that *Assumption 1* somehow.

What does it mean for A to have a win rate of 20%? Suppose we have infinitely many teams, of which A and B are just two. If we arranged a tournament and got all the teams to play against each other then we'd expect A to win 20% of these games. By "expect" here we mean something like if we repeated the entire tournament infinitely often, A would win exactly 20%. Or, given that a tournament has infinitely many teams, we may as well go with precisely 20% of the games in one tournament. Let's make that *Assumption 2*.

I'd like to first approach this problem with a Monte Carlo simulation. "Infinitely often" can be a challenge for computers. So instead, I'll go with a finite number of teams, in this case I'll start with just 6. (You'll see why I picked 6 later.)

So this is the simulation I want to perform: if we have N teams, there are N-2 games of A vs everyone else except B, there are N-2 games of B vs everyone else except A, and 1 game between A and B. We'll assume each game has a 50/50 chance of being a win for either team (Assumption 1). But we're going to condition on the outcome that A wins p (=0.2) of the games and B wins q(=0.6) (Assumption 2). Now you can see why I picked 6, I wanted p(N-1) and q(N-1) to be integers.

Here's a simulation:

In [203]:
N = 6
p = 0.6
q = 0.2
u = 0
v = 0
batch = 1000
for i in xrange(1000):
    # Simulate 1,000,000 games in batches of 1000
    ax = scipy.stats.bernoulli.rvs(0.5, size=(batch, N-2))
    bx = scipy.stats.bernoulli.rvs(0.5, size=(batch, N-2))
    ab = scipy.stats.bernoulli.rvs(0.5, size=batch)
    na = numpy.sum(ax, axis=-1)
    nb = numpy.sum(bx, axis=-1)
    a = na+ab
    b = nb+1-ab

    # Consider only those which have both of the desired outcome
    w = (np.round(a) == np.round(p*(N-1)))*(np.round(b) == np.round(q*(N-1)))
    # Sum number of wins for A vs B
    u += np.sum(w*ab)
    # Count total number of games with proportions p and q
    v += np.sum(w)
print u, v, u/v

47172 55036 0.8571117086997602


You can now try cranking up N. But there's a small problem. Despite simulating 1,000,000 games, for large N the number of games satisfying our condition is small. For example, in 20 games it's relatively rare to get exactly 12 wins for A and exactly 4 wins for B. The method works but we're wasting a bit of CPU time to get inaccurate estimates.

A nice trick here is to use importance sampling. Start with

$\mathbb{E}[X] = \sum_i p(X_i)X_i$

where we're summing over all possible outcomes. We're approximating this in our simulation by using $p(X_i)$ as the probability we use to generate samples, and summing the terms $X_i$ for each sample.

Suppose we have another probability distribution $q$. Then we have

$\mathbb{E}[X] = \sum_i q(X_i)\frac{p(X_i)}{q(X_i)}X_i$

We can interpret this as saying that we can use distribution $q$ to generate our samples as long as we scale each $X_i$ in the sum by a factor of $\frac{p(X_i)}{q(X_i)}$. The reason we'd like to do this is that despite wanting to compute probabilities assuming even odds for each game, we can choose probabilities that make our desired outcomes more likely as long as we scale their contribution appropriately.

In our case we need to scale by $\frac{0.5}{p}$ each time we generated a win for A, $\frac{0.5}{1-p}$ each time we generated a loss for A, $\frac{0.5}{q}$ for each win for B and $\frac{0.5}{1-q}$ for each loss for B. We'll use 50/50 for A vs B though. This will allow us to go up to 21 teams.

In [211]:
def run(p, q):
    m = 21
    u = 0
    v = 0
    batch = 1000
    for i in xrange(1000):
        # Replaced bernoulli with binom assuming it's a 
        # more efficient way of doing same thing
        na = scipy.stats.binom.rvs(m-2, p, size=batch)
        nb = scipy.stats.binom.rvs(m-2, q, size=batch)
        ab = scipy.stats.bernoulli.rvs(0.5, size=batch)
        a = na+ab
        b = nb+1-ab

        weight = (0.5/p)**na*(0.5/(1-p))**(m-2-na)*(0.5/q)**nb*(0.5/(1-q))**(m-2-nb)
        w = weight*(a == p*(m-1))*(b == q*(m-1))
        u += np.sum(w*ab)
        v += np.sum(w)
    return u/v

print run(0.6, 0.2)

0.8570287185405475


Let's tabulate these results for various p and q

In [212]:
p = [0.2, 0.4, 0.6, 0.8]

m1 = [[run(b, a)
       for a in p]
      for b in p]

In [213]:
from IPython.display import display, Markdown
def tabulate(p, m):
    s = ""
    s += "|p\\q|" + "|".join([str(p[j]) for j in range(len(p))]) + "|" + "\n"
    s += "|" + (len(m)+1)*"-|" + "\n"
    for i in range(len(m)):
        s += ("|**%.1f**|" % p[i]) + "|".join(["%.3f" % m[i][j]
                                              for j in range(len(m))]) + "|" + "\n"

    display(Markdown(s))
    
tabulate(p, m1)

|p\q|0.2|0.4|0.6|0.8|
|-|-|-|-|-|
|**0.2**|0.503|0.272|0.143|0.060|
|**0.4**|0.727|0.501|0.309|0.142|
|**0.6**|0.855|0.690|0.504|0.275|
|**0.8**|0.942|0.857|0.726|0.497|


In principle we didn't have to run a simulation. It's not hard to use the binomial theorem to compute the probabilities for any $N$ and then take the limit. But I liked the idea of running a simple simulation that does nothing more than coin tosses so there's no sleight of hand. (And note that importance sampling is just a speed-up. You *could* use just the fair coin version.)

Method 2 (Maximum Entropy)
--------

We can try a maximum entropy approach.

Suppose we have N teams. Let A be team 0 and B be team 1. Let $a_i$ be the probability of $A$ beating team $i$ for $i=2\ldots N-1$. Similarly let $b_i$ be the probability of $B$ beating team $i$ for $i=2\ldots N-1$. And let $c$ be the probability of A beating B. We want to maximise

$c\log c+(1-c)\log(1-c)+\sum_{i=2}^{N-1}\left(a_i\log a_i+(1-a_i)\log (1-a_i)+b_i\log b_i+(1-b_i)\log (1-b_i)\right)$

subject to the constraints

$c+\sum_{i=2}^{N-1} a_i = p(N-1)$ and $1-c+\sum_{i=2}^{N-1} b_i = q(N-1)$.

Introducing Lagrange multipliers define

$L(c,a_i,b_i,\lambda,\mu) = c\log c+(1-c)\log(1-c)+\sum_{i=2}^{N-1}\left(a_i\log a_i+(1-a_i)\log (1-a_i)+b_i\log b_i+(1-b_i)\log (1-b_i)\right)+\lambda(c+\sum_{i=2}^{N-1} a_i-p(N-1))+\mu(1-c+\sum_{i=2}^{N-1} b_i-q(N-1))$

Differentiating with respect to $c$, $a_i$ and $b_i$, setting the results to zero, and rearranging, we get:

$c=\frac{\exp(\mu-\lambda)}{1+\exp{\mu-\lambda}}$,

$a_i = \frac{\exp(-\lambda)}{1+\exp{-\lambda}}$

and 

$b_i = \frac{\exp(-\mu)}{1+\exp{-\mu}}$.

In the limit as N becomes large then the $a_i$ must tend to $p$ and the $b_i$ must tend to $q$. So we can just plug in those values, solve for $\lambda$ and $\mu$, and we get

$c=\frac{p-pq}{p+q-2pq}$. Let's tabulate that:

In [200]:
def f(p, q):
    return (p-p*q)/(p+q-2*p*q)

m2 = [[f(b, a)
       for a in p]
      for b in p]

tabulate(p, m2)

|p\q|0.2|0.4|0.6|0.8|
|-|-|-|-|-|
|**0.2**|0.500|0.273|0.143|0.059|
|**0.4**|0.727|0.500|0.308|0.143|
|**0.6**|0.857|0.692|0.500|0.273|
|**0.8**|0.941|0.857|0.727|0.500|


I hope you're at least mildly surprised that this is almost identical to the previous result.

Now for one last approach.

Method 3 (Bradley-Terry)
-------

Let's use the really simple model of assuming each team $i$ has a strength given by a positive real number $s_i$. The probability of $i$ beating $j$ is assumed to be simply

$\frac{s_i}{s_i+s_j}$

This is known as the [Bradley-Terry](https://en.wikipedia.org/wiki/Bradley–Terry_model) model though it goes back to [Zermelo](https://link.springer.com/article/10.1007%2FBF01180541) in the 20s. It sort of expresses the notion that your strength is the number of opportunities you bring to the game and the formula is the probability that the first opportunity to win is yours.

So team A has strength $s_A$, team B has strength $S_B$, and in the absence of any information about the other teams let's just assign them all an arbitrary strength, say $s_i=1$.

We want $c=\frac{s_A}{s_A+s_b}$ given that $\frac{s_A}{s_A+1}=p$ and $\frac{s_B}{s_B+1}=q$.

Solving for $c$ , we get $c=\frac{p-pq}{p+q-2pq}$.

So we have 3 methods, all seemingy giving the same results.

**Why does Method 2 equal Method 3?**

It seems that the Bradley-Terry model, despite looking like the first thing you might make up, is actually an entropy maximising model, as described in the last paragraph at Wikipedia.

**Why does Method 1 approximately equal Method 2?**

Entropy maximisation methods can be reinterpreted as conditional probabilities in an appropriate limit. A form of this is described in [Campenhout and Cover](http://www-isl.stanford.edu/~cover/papers/transIT/0483camp.pdf).

So, if the only information you have is that A wins $p$ of their games and B wins $q$ then I think $\frac{p-pq}{p+q-2pq}$ is not a bad guess for the probability of A beating B. But this really is a bare minimum model. If you have any other information it needs to be factored in.

One thing I like about Method 1 is that I used only elementary methods. It's nice to see a maximum entropy distribution emerge automatically. I think it can be argued slightly more rigorously that Method 1 is actually an application of [de Finetti's theorem](https://en.wikipedia.org/wiki/De_Finetti%27s_theorem), but that's not something I'm confident to talk about.